In [58]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [59]:
dropColumns=["tbl","citymarketid_1","citymarketid_2","airportid_1","airportid_2","tbl1apk"]
data=pd.read_csv('US Airline Flight Routes and Fares 1993-2024.csv').drop(columns=dropColumns).dropna().reset_index(drop=True)
data=data.loc[(data["city1"]!="Key West, FL") & (data["city2"]!="Key West, FL") & (data["city1"]!="Tulsa, OK") & (data["city2"]!="Tulsa, OK"),:].reset_index(drop=True)
# data=data.loc[(data["Year"]==year) & (data["quarter"]==quarter),:].reset_index(drop=True)
data.head()

C:\Users\alt_user\AppData\Local\Temp\ipykernel_20060\1826946197.py:2: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('US Airline Flight Routes and Fares 1993-2024.csv').drop(columns=dropColumns).dropna().reset_index(drop=True)


,Year,quarter,city1,city2,airport_1,airport_2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2
0,2010,1,"Salt Lake City, UT","Tampa, FL (Metropolitan Area)",SLC,TPA,1887,200,226.59,DL,0.38,247.69,US,0.20,166.99,"Salt Lake City, UT\n(40.758478, -111.888142)","Tampa, FL (Metropolitan Area)\n(37.8606, -78.8..."
1,1998,4,"Colorado Springs, CO","New York City, NY (Metropolitan Area)",COS,HPN,1678,5,280.39,UA,0.73,292.60,NW,0.24,248.27,"Colorado Springs, CO\n(38.835224, -104.819798)","New York City, NY (Metropolitan Area)\n(40.123..."
2,1998,4,"Pittsburgh, PA","Washington, DC (Metropolitan Area)",PIT,BWI,210,152,239.12,US,0.93,245.70,CO,0.03,71.30,"Pittsburgh, PA\n(40.442169, -79.994945)","Washington, DC (Metropolitan Area)\n(38.892062..."
3,2009,3,"Las Vegas, NV","Los Angeles, CA (Metropolitan Area)",LAS,SNA,236,1096,111.74,WN,0.58,118.28,US,0.41,102.75,"Las Vegas, NV\n(36.169202, -115.140597)","Los Angeles, CA (Metropolitan Area)\n(34.05223..."
4,1993,4,"Huntsville, AL","Washington, DC (Metropolitan Area)",HSV,IAD,640,21,242.76,DL,0.48,275.50,UA,0.25,197.69,"Huntsville, AL\n(34.729538, -86.585283)","Washington, DC (Metropolitan Area)\n(38.892062..."


In [60]:
# data=data.sample(50, random_state=42)

In [61]:
G = nx.MultiDiGraph()
GGen=nx.Graph()

dataNp=data["city1"].to_numpy()
posNp=data["Geocoded_City1"].to_numpy()
for airport, pos in zip(dataNp,posNp):
    pos=pos.split("(")[-1][:-1]
    latitude,longtitude=pos.split(",")
    pos=(float(latitude),float(longtitude))
    G.add_node(airport,pos=pos)
    G.add_node(airport,pos=pos)


dataNp=data["city2"].to_numpy()
posNp=data["Geocoded_City2"].to_numpy()
for airport, pos in zip(dataNp,posNp):
    pos=pos.split("(")[-1][:-1]
    latitude,longtitude=pos.split(",")
    pos=(float(latitude),float(longtitude))
    G.add_node(airport,pos=pos)
    GGen.add_node(airport,pos=pos)

In [62]:
# flights=[]

dfNp=data.to_numpy()

for flight in dfNp:
    origin, dest, distance, fare, originPos, destPos = flight[2], flight[3], flight[6], flight[8], flight[-2], flight[-1]
    # flights.append((origin,dest,distance,fare,originPos,destPos))
    G.add_edge(origin,dest,weight=distance,price=fare)
    GGen.add_edge(origin,dest,weight=distance,price=fare)


In [63]:

if(input("Do you want random start and end cities? (y/n) ")=="y"):
    start= data.sample(1)["city1"].values[0]
    end= data.sample(1)["city2"].values[0]

else:
    start = 'Atlanta, GA (Metropolitan Area)'
    end = 'Eugene, OR'
print("Start: ",start," End: ",end)

Start:  New York City, NY (Metropolitan Area)  End:  Tucson, AZ


In [ ]:
from FunctionsGenetic import genetic_algorithm
# Genetic Algorithm to find the best route between two cities

# Start:  Chicago, IL  End:  New York City, NY (Metropolitan Area)
import time

start_time = time.time()

route = genetic_algorithm(GGen, start, end, sampleSize=100, parentCount=10, generations=100)
print("--- %s seconds Genetic Algorithm---" % (time.time() - start_time))

route[-1][-1]

child Count 44
mutation
mutation
child Count 40
child Count 78
child Count 10
mutation
child Count 8
child Count 30
mutation
child Count 34
mutation
child Count 8
child Count 30
child Count 16
--- 0.7358388900756836 seconds Genetic Algorithm---


2420

In [ ]:
path=[]
for node in route:
    path.append(node[0])
path

['New York City, NY (Metropolitan Area)',
 'Richmond, VA',
 'Tampa, FL (Metropolitan Area)']

In [65]:
from Functions import heuristic, dijkstra, a_star

start_time = time.time()

print("Dijkstra") 
distances, preds = dijkstra(G, start)
if(distances[end]==float('inf')):
    print("No path found")
else:
    path = []
    node = end
    while node is not None:
        path.append(node)
        node = preds[node]
    path.reverse()
    print(f"Optimal Flight Path: {path}, Distance with heuristic: {distances[end]} km")
print("--- %s seconds Dijkstra's---" % (time.time() - start_time))

start_time = time.time()

# #####################
print("A*")
distances, preds = a_star(G, start, end, heuristic)

path = []
node = end
while node is not None:
    path.append(node)
    node = preds[node]
path.reverse()
# path = nx.astar_path(G, start, end, heuristic=heuristic, weight='weight')
print(f"Optimal Flight Path: {path}, Distance with heuristic: {distances[end]} km")
print("--- %s seconds A*---" % (time.time() - start_time))

Dijkstra
Optimal Flight Path: ['New York City, NY (Metropolitan Area)', 'Tucson, AZ'], Distance with heuristic: 2142.0 km
--- 0.03238058090209961 seconds Dijkstra's---
A*
Optimal Flight Path: ['New York City, NY (Metropolitan Area)', 'Tucson, AZ'], Distance with heuristic: 2142.0 km
--- 0.012818098068237305 seconds A*---


In [66]:
from Functions import multi_criteria_a_star, reconstruct_path

print("Multi A*")
start_time = time.time()
pareto_front, predecessors = multi_criteria_a_star(G, start, end, heuristic)
paths = reconstruct_path(start, end, pareto_front, predecessors)

# Print results
for path, (distance, money) in paths:
    print(f"Path: {path}, Distance: {distance}, Money: {money}")
print("--- %s seconds A*---" % (time.time() - start_time))

Multi A*
Path: ['New York City, NY (Metropolitan Area)', 'Tucson, AZ'], Distance: 2173, Money: 152.1
Path: ['New York City, NY (Metropolitan Area)', 'Tucson, AZ'], Distance: 2142, Money: 230.11
--- 0.05200314521789551 seconds A*---
